In [2]:
from flask import Flask, request, jsonify
import tensorflow as tf
import pickle
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Initialize the Flask app
app = Flask(__name__)

# Load the model
MODEL_PATH = 'crop_yield_model.h5'
model = tf.keras.models.load_model(MODEL_PATH)

# Load the label encoders
with open('label_encoders.pkl', 'rb') as encoder_file:
    label_encoders = pickle.load(encoder_file)

# Define a function to preprocess the input image
def preprocess_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image /= 255.0  # Rescale image
    return image

# Define a route for predictions
@app.route('/predict', methods=['POST'])
def predict():
    # Ensure an image file was provided
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    # Get the file from the request
    file = request.files['file']

    # Save the file to a temporary location
    file_path = r"C:\Life Projects\DAIICT_hackout\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid\Tomato___Bacterial_spot\0a6d40e4-75d6-4659-8bc1-22f47cdb2ca8___GCREC_Bact.Sp 6247.JPG"
    file.save(file_path)

    # Preprocess the image
    processed_image = preprocess_image(file_path, target_size=(224, 224))

    # Make the prediction
    prediction = model.predict(processed_image)
    predicted_class = np.argmax(prediction, axis=1)

    # Decode the predicted class using the label encoders
    inverse_label_map = {v: k for k, v in label_encoders.items()}
    predicted_label = inverse_label_map[predicted_class[0]]

    # Return the prediction as JSON
    return jsonify({'predicted_class': predicted_label, 'confidence': float(np.max(prediction))})

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\CHINMAY\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
